In [24]:
import logging
import gensim
from gensim.models.word2vec import LineSentence

In [28]:
with open(data_file, 'w') as file:
    file.writelines('ss jk lio j' + '\n')
    file.writelines('ss df fds' + '\n')
    file.writelines('ss $$$ fds' + '\n')

In [15]:
'　'

'\u3000'

In [29]:
data_file = 'data_gen/test'
dim = 2
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = gensim.models.Word2Vec(
    sentences=LineSentence(data_file),
    size=dim,
    min_count=1,
    iter=2
)

2018-09-10 09:07:25,112 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-09-10 09:07:25,113 : INFO : collecting all words and their counts
2018-09-10 09:07:25,113 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-09-10 09:07:25,114 : INFO : collected 7 word types from a corpus of 10 raw words and 3 sentences
2018-09-10 09:07:25,114 : INFO : Loading a fresh vocabulary
2018-09-10 09:07:25,114 : INFO : min_count=1 retains 7 unique words (100% of original 7, drops 0)
2018-09-10 09:07:25,115 : INFO : min_count=1 leaves 10 word corpus (100% of original 10, drops 0)
2018-09-10 09:07:25,115 : INFO : deleting the raw counts dictionary of 7 items
2018-09-10 09:07:25,115 : INFO : sample=0.001 downsamples 7 most-common words
2018-09-10 09:07:25,115 : INFO : downsampling leaves estimated 0 word corpus (8.8% of prior 10)
2018-09-10 09:07:25,116 : INFO : estimated required memory for 7 words and 2 dimensions: 3612 bytes
2018-09-10 0

In [32]:
import jieba
from jieba import posseg

In [56]:
s = '今天天气还是不错的'
posseg.lcut(s, HMM=False)

[pair('今天天气', 'i'), pair('还是', 'c'), pair('不错', 'a'), pair('的', 'uj')]

In [43]:
a , b = list(zip(*posseg.lcut(s, HMM=False)))

In [47]:
import nltk

In [49]:
s = 'Today is good.'
s = nltk.word_tokenize(s)
nltk.pos_tag(s)

[('Today', 'NN'), ('is', 'VBZ'), ('good', 'JJ'), ('.', '.')]

In [68]:
s = 'Today is good.'
split_word_en(s, True)

(['Today', 'is', 'good', '.'], ['NN', 'VBZ', 'JJ', '.'])

In [67]:
def split_word_en(s, have_tag=False):
    """
    英文分词
    :param s: str
    :return:
    """
    if have_tag is False:
        s = nltk.word_tokenize(s)
        return s
    else:
        s = nltk.word_tokenize(s)
        _, tags = list(zip(*nltk.pos_tag(s)))
        return s, list(tags)